In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string 
from nltk.stem import WordNetLemmatizer

In [2]:
import pandas as pd
df1 = pd.read_csv("gilettesent4.csv", encoding='ISO-8859-1')

In [3]:
df1.head()

,Tweets,Label
0,Barbasol Once Showed Gillette How To Make a Co...,Negative
1,BoycottGillette,Negative
2,MeetBarbasol arbasol Once Showed Gillette How ...,Negative
3,BoycottGillette MeetBarbasol destinyisbright C...,Negative
4,MeetBarbasol MeetBarbasol Barbasol Once Showed...,Negative


In [4]:
Tweet = []
Labels = []

for row in df1["Tweets"]:
    #tokenize words
    words = word_tokenize(row)
    #remove punctuations
    clean_words = [word.lower() for word in words if word not in set(string.punctuation)]
    #remove stop words
    english_stops = set(stopwords.words('english'))
    characters_to_remove = ["''",'``',"rt","https","’","“","”","\u200b","--","n't","'s","...","//t.c","'re" ,"'m"]
    clean_words = [word for word in clean_words if word not in english_stops]
    clean_words = [word for word in clean_words if word not in set(characters_to_remove)]
    #Lematise words
    wordnet_lemmatizer = WordNetLemmatizer()
    lemma_list = [wordnet_lemmatizer.lemmatize(word) for word in clean_words]
    Tweet.append(lemma_list)

    for row in df1["Label"]:
        Labels.append(row)

In [5]:
combined = zip(Tweet, Labels)

In [6]:
def bag_of_words(words):
    return dict([(word, True) for word in words])

In [7]:
Final_Data = []
for r, v in combined:
    bag_of_words(r)
    Final_Data.append((bag_of_words(r),v))

In [8]:
import random
random.shuffle(Final_Data)
print(len(Final_Data))

265


In [9]:
train_set, test_set = Final_Data[0:188], Final_Data[188:]

import nltk
import collections
from nltk.metrics.scores import (accuracy, precision, recall, f_measure) 
from nltk import metrics

refsets = collections. defaultdict(set)
testsets = collections.defaultdict(set)

classifier = nltk.NaiveBayesClassifier.train(train_set)
 
for i, (feats, label) in enumerate(test_set):
    refsets[label].add(i)
    observed = classifier.classify(feats)
    testsets[observed].add(i)

print("Naive Bayes Performance with Unigrams ")    
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))

Naive Bayes Performance with Unigrams 
Accuracy: 0.6883116883116883


In [10]:
nb_classifier = nltk.NaiveBayesClassifier.train(train_set)

nbrefset = collections.defaultdict(set)
nbtestset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    nbrefset[label].add(i)
    observed = nb_classifier.classify(feats)
    nbtestset[observed].add(i)
    
print("UnigramNB Results")
print('Brand Positive Precision:', precision(refsets['Positive'], testsets['Positive']))
print('Brand Positive Recall:', recall(refsets['Positive'], testsets['Positive']))
print('Brand Positive F-measure:', f_measure(refsets['Positive'], testsets['Positive']))
print('Brand Damaging Precision:', precision(refsets['Negative'], testsets['Negative']))
print('Brand Damaging Recall:', recall(nbtestset['Negative'], nbrefset['Negative']))
print('Brand Damaging F-measure:', f_measure(refsets['Negative'], testsets['Negative']))
print("")

UnigramNB Results
Brand Positive Precision: 0.6
Brand Positive Recall: 0.75
Brand Positive F-measure: 0.6666666666666666
Brand Damaging Precision: 0.7837837837837838
Brand Damaging Recall: 0.7837837837837838
Brand Damaging F-measure: 0.7073170731707318



In [11]:
classifier.show_most_informative_features(n=50)

Most Informative Features
                    love = True           Positi : Negati =      5.8 : 1.0
               'gillette = True           Positi : Negati =      4.0 : 1.0
                    away = True           Positi : Negati =      4.0 : 1.0
              boycotting = True           Positi : Negati =      4.0 : 1.0
                    look = True           Positi : Negati =      4.0 : 1.0
                     guy = True           Positi : Negati =      4.0 : 1.0
                   video = True           Positi : Negati =      4.0 : 1.0
                    real = True           Negati : Positi =      3.7 : 1.0
                     get = True           Negati : Positi =      3.5 : 1.0
                      ad = True           Positi : Negati =      3.2 : 1.0
                    hear = True           Positi : Negati =      3.1 : 1.0
                   every = True           Positi : Negati =      3.1 : 1.0
                     son = True           Positi : Negati =      3.1 : 1.0

In [12]:
from nltk.classify import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier.train(train_set, 
                                             binary=True, 
                                             entropy_cutoff=0.8, 
                                             depth_cutoff=5, 
                                             support_cutoff=30)
refset = collections.defaultdict(set)
testset = collections.defaultdict(set)
 
for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = dt_classifier.classify(feats)
    testset[observed].add(i)

print("UnigramDT Results")
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))
print('Brand Positive Precision:', precision(refsets['Positive'], testsets['Positive']))
print('Brand Positive Recall:', recall(refsets['Positive'], testsets['Positive']))
print('Brand Positive F-measure:', f_measure(refsets['Positive'], testsets['Positive']))
print('Brand Damaging Precision:', precision(refsets['Negative'], testsets['Negative']))
print('Brand Damaging Recall:', recall(nbtestset['Negative'], nbrefset['Negative']))
print('Brand Damaging F-measure:', f_measure(refsets['Negative'], testsets['Negative']))
print("")

UnigramDT Results
Accuracy: 0.6883116883116883
Brand Positive Precision: 0.6
Brand Positive Recall: 0.75
Brand Positive F-measure: 0.6666666666666666
Brand Damaging Precision: 0.7837837837837838
Brand Damaging Recall: 0.7837837837837838
Brand Damaging F-measure: 0.7073170731707318



In [13]:
from nltk.classify import MaxentClassifier

logit_classifier = MaxentClassifier.train(train_set, algorithm='gis', trace=0, max_iter=10, min_lldelta=0.5)

for i, (feats, label) in enumerate(test_set):
    refset[label].add(i)
    observed = logit_classifier.classify(feats)
    testset[observed].add(i)
    
print("UnigramsLogit Results")
print("Accuracy:",nltk.classify.accuracy(classifier, test_set))
print('Brand Positive Precision:', precision(refsets['Positive'], testsets['Positive']))
print('Brand Positive Recall:', recall(refsets['Positive'], testsets['Positive']))
print('Brand Positive F-measure:', f_measure(refsets['Positive'], testsets['Positive']))
print('Brand Damaging Precision:', precision(refsets['Negative'], testsets['Negative']))
print('Brand Damaging Recall:', recall(nbtestset['Negative'], nbrefset['Negative']))
print('Brand Damaging F-measure:', f_measure(refsets['Negative'], testsets['Negative']))
print("")
 

UnigramsLogit Results
Accuracy: 0.6883116883116883
Brand Positive Precision: 0.6
Brand Positive Recall: 0.75
Brand Positive F-measure: 0.6666666666666666
Brand Damaging Precision: 0.7837837837837838
Brand Damaging Recall: 0.7837837837837838
Brand Damaging F-measure: 0.7073170731707318

